In [0]:
%run /Workspace/Users/mahatoruchita712@gmail.com/PySpark-use-cases/use_case1/utils

In [0]:
df_gold = read_file("silver", "orders", "delta", "failfast")

In [0]:
df_gold.display()

## Creating fact tables


### Fact Table : Orders

In [0]:
fact_orders = df_gold.select(
              col("order_id"),
              col("customer_id"),
              col("Order_date"),
              col("rder_date"),
              col("status")            
)

fact_orders = write_file(fact_orders, "gold", "fact_orders", "overwrite")

### Fact Table : Order items

In [0]:
fact_order_items = df_gold.select(
                   col("order_id"),
                   col("items_product_id"),
                   col("items_product_name"),
                   col("items_quantity"),
                   col("items_price"),
                   col("Total_amount")
)

fact_order_items = write_file(fact_order_items, "gold", "fact_order_items", "overwrite")


## Creating dimension tables

### Dimension Table : Customers

In [0]:
dim_customers = df_gold.select(
                  col("customer_id"),
                  col("customer_name"),
                  col("customer_email")
)

dim_customers = write_file(dim_customers, "gold", "dim_customers", "overwrite")

### Dimension Table : Products

In [0]:
dim_products = df_gold.select(
                  col("items_product_id"),
                  col("items_product_name")
)

dim_products = write_file(dim_products, "gold", "dim_products", "overwrite")

### Dimension Table : Order status

In [0]:
dim_order_status = df_gold.select(
                  col("status")
)

dim_order_status = write_file(dim_order_status, "gold", "dim_order_status", "overwrite")

### Dimension Table : Location

In [0]:
dim_location = df_gold.select(
                  col("shipping_address_city").alias("City"),
                  col("shipping_address_state").alias("State"),
                  col("shipping_address_country").alias("Country")
)

dim_location = write_file(dim_location, "gold", "dim_location", "overwrite")

## Getting the top selling products

In [0]:
fact_order_items = read_file("gold", "fact_order_items", "delta", "failfast")

top_selling_product_df = fact_order_items \
                           .groupBy("items_product_name") \
                           .agg(sum("items_quantity").alias("total_quantity_sold")) \
                           .orderBy(col("total_quantity_sold").desc()) \
                           .limit(1)

display(top_selling_product_df)

In [0]:
"""
top_selling_df = df_gold.groupBy("items_product_name") \
                        .agg(sum("items_quantity").alias("total_quantity_sold")) \
                        .orderBy(col("total_quantity_sold").desc()) \
                        .dropna()
top_selling_df.display()

df_gold = write_file(df_gold, "gold", "top_selling", "append")
"""

## Getting the Average order value per customer

In [0]:
customer_catalog.gold.fact_orders

In [0]:
avg_order_value_per_cus_df = df_gold.groupBy("customer_id") \
                               .agg(avg("Total_amount").alias("avg_order_value")) \
                               .orderBy(col("avg_order_value").desc()) \
                               .dropna()
avg_order_value_per_cus_df.display()

df_gold = write_file(df_gold, "gold", "avg_order_per_cus", "append")

## Getting overall average order value

In [0]:
avg_order_value_df = df_gold.dropna(how='any', subset=['Total_amount']) \
                         .agg(avg("Total_amount").alias("avg_order_value")) 

avg_order_value_df.display()

df_gold = write_file(df_gold, "gold", "overall_avg_order_value", "append")

## Getting number of order per city

In [0]:
order_per_city_df = df_gold.groupBy("shipping_address_city") \
                         .agg(count("order_id").alias("total_orders")) \
                         .dropna()

order_per_city_df.display()

df_gold = write_file(df_gold, "gold", "order_per_city", "append")

## Order status distribution

In [0]:
order_distribution_df = df_gold.groupBy("status") \
                                .agg(count("order_id").alias("total_orders"))

order_distribution_df.display()

df_gold = write_file(df_gold, "gold", "order_distribution", "append")